In [1]:
pip install pandas openpyxl ccxt numpy matplotlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.0/628.0 kB 30.8 MB/s eta 0:00:00


In [1]:
from google.colab import files
uploaded = files.upload()

Saving BTC_spot_ETF_data_20250709.xlsx to BTC_spot_ETF_data_20250709 (2).xlsx


In [2]:
import pandas as pd
import ccxt
import numpy as np
import traceback
import io # Required for reading uploaded file data
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# --- Configuration Parameters ---
INITIAL_CAPITAL = 100000.0
POSITION_INVESTMENT_AMOUNT = INITIAL_CAPITAL * 0.25
TRANSACTION_COST_PCT = 0.001

ETF_DATA_FILE_PATH = 'BTC_spot_ETF_data_20250618.xlsx' # IMPORTANT: Update this path
BTC_SYMBOL = 'BTC/USD'
EXCHANGE_NAME = 'coinbase'

In [3]:
# --- Data Loading and Preparation Functions (no changes) ---
def load_etf_data(file_source):
    try:
        df = pd.read_excel(file_source)
        if 'Date' not in df.columns or 'NetInflow' not in df.columns:
            raise ValueError("Excel file must contain 'Date' and 'NetInflow' columns.")
        df['Date'] = pd.to_datetime(df['Date'])
        df = df.sort_values(by='Date')
        df.set_index('Date', inplace=True)
        df = df.resample('D').sum()
        return df
    except Exception as e:
        print(f"Error loading ETF data: {e}")
        return None

def fetch_btc_price_data_full(symbol, exchange_name, start_date_str, end_date_str):
    exchange = getattr(ccxt, exchange_name)()
    if not exchange.has['fetchOHLCV']: return None
    all_ohlcv = []
    since = exchange.parse8601(start_date_str + 'T00:00:00Z')
    end_timestamp = exchange.parse8601(end_date_str + 'T23:59:59Z')
    limit = 1000
    while since < end_timestamp:
        try:
            print(f"Fetching BTC data from {exchange_name} starting from {exchange.iso8601(since)}...")
            ohlcv = exchange.fetch_ohlcv(symbol, '1d', since, limit)
            if not ohlcv: break
            ohlcv = [c for c in ohlcv if c[0] <= end_timestamp]
            if not ohlcv: break
            all_ohlcv.extend(ohlcv)
            since = ohlcv[-1][0] + 86400000
            time.sleep(exchange.rateLimit / 1000)
        except Exception as e:
            print(f"An error occurred while fetching BTC data: {e}"); traceback.print_exc(); return None
    if not all_ohlcv: return None
    df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'Open', 'High', 'Low', 'Close', 'Volume'])
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms')
    df = df.drop_duplicates(subset='timestamp').sort_values(by='Date')
    df.set_index('Date', inplace=True)
    return df[['Open', 'High', 'Low', 'Close', 'Volume']]

def align_data(etf_df, btc_df):
    if etf_df is None or btc_df is None: return None
    if etf_df.index.tz is not None: etf_df.index = etf_df.index.tz_localize(None)
    if btc_df.index.tz is not None: btc_df.index = btc_df.index.tz_localize(None)
    combined_df = pd.merge(btc_df, etf_df, left_index=True, right_index=True, how='inner')
    combined_df['NetInflow_shifted'] = combined_df['NetInflow'].shift(1)
    combined_df.dropna(subset=['NetInflow_shifted'], inplace=True)
    return combined_df

In [4]:
# --- NEW SYMMETRIC SIGNAL GENERATION FUNCTION ---
def generate_signals_symmetric_ma(df, ma_window):
    """
    Generates trading signals based on a single, symmetric moving average crossover.
    - Buy Signal (1): NetInflow > N-day MA of NetInflow
    - Sell Signal (-1): NetInflow < N-day MA of NetInflow
    """
    df_signal = df.copy()

    # Calculate the single moving average for the signal threshold
    df_signal['Threshold_MA'] = df_signal['NetInflow_shifted'].rolling(window=ma_window).mean()

    # Drop rows where the MA is not yet available
    df_signal.dropna(subset=['Threshold_MA'], inplace=True)

    # Initialize Signal column
    df_signal['Signal'] = 0

    # Generate Buy Signal
    buy_condition = (df_signal['NetInflow_shifted'] > df_signal['Threshold_MA'])
    df_signal.loc[buy_condition, 'Signal'] = 1

    # Generate Sell Signal
    sell_condition = (df_signal['NetInflow_shifted'] < df_signal['Threshold_MA'])
    df_signal.loc[sell_condition, 'Signal'] = -1

    return df_signal

# --- Backtesting and Performance Functions (no changes) ---
def run_backtest(df, initial_capital, position_investment, transaction_cost_pct):
    cash = initial_capital
    btc_held = 0.0
    position_open = False
    entry_price = 0.0
    portfolio_values = []
    num_trades = 0
    num_wins = 0
    for i in range(len(df)):
        date = df.index[i]
        today_open = df['Open'].iloc[i]
        today_close = df['Close'].iloc[i]
        signal = df['Signal'].iloc[i]
        if not position_open and signal == 1:
            if cash >= position_investment:
                buy_price = today_open
                btc_to_acquire = position_investment / buy_price
                fee = position_investment * transaction_cost_pct
                cash -= (position_investment + fee)
                btc_held = btc_to_acquire
                position_open = True
                entry_price = buy_price
        elif position_open and signal == -1:
            sell_price = today_open
            proceeds = btc_held * sell_price
            fee = proceeds * transaction_cost_pct
            cash += (proceeds - fee)
            btc_held = 0.0
            position_open = False
            num_trades += 1
            if sell_price > entry_price:
                num_wins += 1
            entry_price = 0.0
        current_portfolio_value = cash + (btc_held * today_close)
        portfolio_values.append({'Date': date, 'PortfolioValue': current_portfolio_value})
    portfolio_df = pd.DataFrame(portfolio_values).set_index('Date') if portfolio_values else None
    return {'portfolio_df': portfolio_df, 'num_trades': num_trades, 'num_wins': num_wins}

def calculate_performance_metrics(portfolio_df, initial_capital):
    if portfolio_df is None or portfolio_df.empty or len(portfolio_df) < 2:
        return {'Sharpe Ratio': -99, 'Total Return (%)': -100, 'Max Drawdown (%)': -100}
    metrics = {}
    final_value = portfolio_df['PortfolioValue'].iloc[-1]
    metrics['Total Return (%)'] = ((final_value / initial_capital) - 1) * 100
    portfolio_df['Peak'] = portfolio_df['PortfolioValue'].cummax()
    portfolio_df['Drawdown'] = (portfolio_df['PortfolioValue'] - portfolio_df['Peak']) / portfolio_df['Peak']
    metrics['Max Drawdown (%)'] = portfolio_df['Drawdown'].min() * 100
    portfolio_df['Daily Return'] = portfolio_df['PortfolioValue'].pct_change()
    avg_daily_return = portfolio_df['Daily Return'].mean()
    std_daily_return = portfolio_df['Daily Return'].std()
    if std_daily_return != 0 and not np.isnan(std_daily_return):
        metrics['Sharpe Ratio'] = (avg_daily_return / std_daily_return) * np.sqrt(252)
    else:
        metrics['Sharpe Ratio'] = 0.0
    return metrics

def calculate_buy_and_hold_performance(df, initial_capital, transaction_cost_pct):
    if df is None or df.empty: return None
    buy_price = df['Open'].iloc[0]
    final_price = df['Close'].iloc[-1]
    fee = initial_capital * transaction_cost_pct
    amount_to_invest = initial_capital - fee
    btc_bought = amount_to_invest / buy_price
    portfolio_df = pd.DataFrame(index=df.index)
    portfolio_df['PortfolioValue'] = btc_bought * df['Close']
    num_trades = 1
    num_wins = 1 if final_price > buy_price else 0
    return {'portfolio_df': portfolio_df, 'num_trades': num_trades, 'num_wins': num_wins}

# --- Main Execution Block ---
if __name__ == '__main__':
    file_source = None
    try:
        from google.colab import files
        print("Running in a Colab environment. Please upload your file.")
        uploaded = files.upload()
        if uploaded:
            filename = next(iter(uploaded))
            print(f'User uploaded file "{filename}"')
            file_source = io.BytesIO(uploaded[filename])
        else:
            print("No file was uploaded."); exit()
    except (ImportError, ModuleNotFoundError):
        print("Not in Colab. This script is designed for Google Colab. Exiting."); exit()

    print("\nLoading data...")
    etf_df = load_etf_data(file_source)
    if etf_df is None: exit()

    start_date_btc = etf_df.index.min().strftime('%Y-%m-%d')
    end_date_btc = etf_df.index.max().strftime('%Y-%m-%d')

    btc_df = fetch_btc_price_data_full(BTC_SYMBOL, EXCHANGE_NAME, start_date_btc, end_date_btc)
    if btc_df is None: exit()

    base_data = align_data(etf_df, btc_df)
    if base_data is None or base_data.empty:
        print("Could not align data. Exiting."); exit()

    print("\n--- Calculating Buy & Hold Benchmark Performance ---")
    benchmark_results = calculate_buy_and_hold_performance(base_data, INITIAL_CAPITAL, TRANSACTION_COST_PCT)
    benchmark_metrics = calculate_performance_metrics(benchmark_results['portfolio_df'], INITIAL_CAPITAL)
    print("Benchmark Metrics Calculated.")

    # --- MODIFIED: Optimization loop for the new symmetric strategy ---
    ma_window_options = range(3, 21) # Test windows from 3 to 20 days
    optimization_results = []

    print("\n--- Starting Symmetric MA Strategy Optimization ---")
    for days in ma_window_options:
        print(f"Testing with MA Window: {days} days")
        df_with_signals = generate_signals_symmetric_ma(base_data.copy(), ma_window=days)
        if df_with_signals.empty: continue

        backtest_results = run_backtest(df_with_signals, INITIAL_CAPITAL, POSITION_INVESTMENT_AMOUNT, TRANSACTION_COST_PCT)
        portfolio_history = backtest_results['portfolio_df']
        num_trades = backtest_results['num_trades']
        num_wins = backtest_results['num_wins']

        if portfolio_history is not None:
            metrics = calculate_performance_metrics(portfolio_history, INITIAL_CAPITAL)
            win_rate = (num_wins / num_trades) * 100 if num_trades > 0 else 0
            result_entry = {
                'MA Window': days, # Simplified parameter
                'Sharpe Ratio': metrics['Sharpe Ratio'], 'Total Return (%)': metrics['Total Return (%)'],
                'Max Drawdown (%)': metrics['Max Drawdown (%)'], 'Num Trades': num_trades, 'Win Rate (%)': win_rate
            }
            optimization_results.append(result_entry)

    if optimization_results:
        results_df = pd.DataFrame(optimization_results)
        print("\n\n--- OPTIMIZATION COMPLETE ---")
        results_df_sorted = results_df.sort_values(by='Sharpe Ratio', ascending=False)
        results_df_sorted['Win Rate (%)'] = results_df_sorted['Win Rate (%)'].map('{:.1f}'.format)
        print(results_df_sorted.to_string(index=False))

        best_params_row = results_df_sorted.iloc[0]

        benchmark_win_rate = (benchmark_results['num_wins'] / benchmark_results['num_trades']) * 100 if benchmark_results['num_trades'] > 0 else 0
        comparison_data = {
            'Metric': ['Sharpe Ratio', 'Total Return (%)', 'Max Drawdown (%)', 'Num Trades', 'Win Rate (%)'],
            'Best Strategy': [
                float(best_params_row['Sharpe Ratio']), float(best_params_row['Total Return (%)']),
                float(best_params_row['Max Drawdown (%)']), int(best_params_row['Num Trades']),
                float(best_params_row['Win Rate (%)'])
            ],
            'Buy & Hold': [
                benchmark_metrics['Sharpe Ratio'], benchmark_metrics['Total Return (%)'],
                benchmark_metrics['Max Drawdown (%)'], benchmark_results['num_trades'],
                benchmark_win_rate
            ]
        }
        comparison_df = pd.DataFrame(comparison_data)

        print("\n\n--- Best Strategy vs. Buy & Hold Benchmark ---")
        test_start_date = base_data.index.min().strftime('%Y-%m-%d')
        test_end_date = base_data.index.max().strftime('%Y-%m-%d')
        print(f"Test Period for Both: {test_start_date} to {test_end_date}")
        print(f"Best Strategy Parameter: MA Window = {int(best_params_row['MA Window'])} days")
        comparison_df['Best Strategy'] = comparison_df['Best Strategy'].map('{:.2f}'.format)
        comparison_df['Buy & Hold'] = comparison_df['Buy & Hold'].map('{:.2f}'.format)
        print(comparison_df.to_string(index=False))

        # --- NEW VISUALIZATION: Line chart for single-parameter optimization ---
        print("\n--- Generating Optimization Line Chart ---")
        plt.figure(figsize=(14, 7))
        plt.plot(results_df['MA Window'], results_df['Sharpe Ratio'], marker='o', linestyle='-', color='b')
        plt.title('Optimization of Sharpe Ratio by MA Window', fontsize=16)
        plt.xlabel('Moving Average Window (Days)', fontsize=12)
        plt.ylabel('Sharpe Ratio', fontsize=12)
        plt.xticks(results_df['MA Window'])
        plt.grid(True, which='both', linestyle='--', linewidth=0.5)
        plt.show()

    else:
        print("\nOptimization did not yield any results.")

Running in a Colab environment. Please upload your file.


Saving BTC_spot_ETF_data_20250709.xlsx to BTC_spot_ETF_data_20250709 (3).xlsx
User uploaded file "BTC_spot_ETF_data_20250709 (3).xlsx"

Loading data...
Fetching BTC data from coinbase starting from 2024-01-10T00:00:00.000Z...
An error occurred while fetching BTC data: name 'time' is not defined
Could not align data. Exiting.

--- Calculating Buy & Hold Benchmark Performance ---


Traceback (most recent call last):
  File "/tmp/ipython-input-3-1238726463.py", line 32, in fetch_btc_price_data_full
    time.sleep(exchange.rateLimit / 1000)
    ^^^^
NameError: name 'time' is not defined


TypeError: 'NoneType' object is not subscriptable